Use this notebook to check if a table or field has been used in a report

Written by: Meagan Longoria

Last Updated: 5 Nov 2024

Pre-reqs: You must be able to install the sempy_labs package in your environment or have an environment available with it already installed. Instructions for loading into a custom environment can be found [here](https://github.com/microsoft/semantic-link-labs?tab=readme-ov-file#load-semantic-link-labs-into-a-custom-fabric-environment). If you create a custom environment, you can comment out the next cell (which contains the install command).   

Limitations: The report you search must use the PBIR format.

Instructions: 
1. If you have semantic-link-labs package pre-installed, skip the first code cell below. Otherwise, run the first code cell.
2. Run the second code and third code cells.
4. Enter the input values for the widgets below the third code cell. 
5. Run the fourth code cell. The output will be a table sorted by table name and field name.

In [ ]:
# Comment out or skip if you have a custom environment with semantic-link-labs pre-installed
#%pip install semantic-link-labs

In [ ]:
import sempy_labs as labs
from sempy_labs import report as rep
from sempy_labs.report import ReportWrapper

import ipywidgets as widgets
import pandas as pd

In [ ]:
# Run this cell
# Then enter at least the report name and workspace name. You may also enter a table name or a table name and a field name to further filter the list.
w_report = widgets.Text(value='', description='Report:');
w_workspace = widgets.Text(value = '', description = 'Workspace:'); 
w_table = widgets.Text(value='', description='Table:');
w_field = widgets.Text(value='', description='Field:');
w_validfield = widgets.Dropdown(options=['all fields','valid fields only', 'invalid fields only' ],
    value='all fields',
    description='Invalid Fields:',
)
display(w_report)
display(w_workspace)
display(w_table)
display(w_field)
display(w_validfield)

In [ ]:
# Run this cell AFTER entering inputs into the widgets above.
int_option = 0

if w_report.value == '' or w_workspace.value =='':
    print ("Please enter a report and workspace")
else: 
    rpt = ReportWrapper(report=w_report.value, workspace=w_workspace.value)
    df_smobjects = rpt.list_semantic_model_objects(extended=True)
    if w_table.value == '' and w_field.value == '':
        int_option = 1
        df_result = df_smobjects.sort_values(["Table Name", "Object Name"])
    elif w_table.value == '' and w_field.value != '': 
        int_option = 2
        print ('Please enter the table that contains the field for which you want to search.')
    elif w_table.value !='' and w_field.value == '': 
        int_option = 3
        df_result = df_smobjects[df_smobjects['Table Name'] == w_table.value].sort_values(["Table Name", "Object Name"]) 
    else:
        int_option = 4
        df_result= df_smobjects.loc[df_smobjects['Table Name'].eq(w_table.value) & df_smobjects['Object Name'].eq(w_field.value)].sort_values(["Table Name", "Object Name"])
if int_option == 1: 
    print("No table or field filters")
elif int_option == 3: 
    print("Visuals that use fields from the " + w_table.value + " table")
elif int_option == 4: 
    print("Visuals that use the '" + w_table.value + "'[" + w_field.value + "] field")
else: 
    print("Unknown selection")

if w_validfield.value == "all fields":
    print("Valid and invalid fields")
    #display(df_result)
elif w_validfield.value == "valid fields only":
    df_valfield = df_result[df_result['Valid Semantic Model Object'] == True]
    print("Valid fields only")
    df_result = df_valfield
    #display(df_valfield)
elif w_validfield.value == "invalid fields only":
    df_valfield = df_result[df_result['Valid Semantic Model Object'] == False]
    print("Invalid fields only")
    df_result = df_valfield
    #display(df_valfield)
else: 
    print("Selection error on invalid fields")

df_viz = rpt.list_visuals()
df_viz["Report Source Object"] = "'" + df_viz["Page Display Name"] + "'[" + df_viz["Visual Name"] + "]" 
df_viz = df_viz.loc[:, ['Page Name', 'Page Display Name','Visual Name','Display Type', 'X','Y','Z','Report Source Object']]
df_final = pd.merge(df_result, df_viz, how='left', on=['Report Source Object','Report Source Object'])
df_final.rename(columns={'X': 'Visual Loc X', 'Y': 'Visual Loc Y', 'Z': 'Visual Loc Z'}, inplace=True)
display(df_final)